In [81]:
import re
import pandas as pd
import numpy as np

## Как изменить формат телефона для сопоставления двух массивов.

У нас есть задание от банка, для которого нужно соеденить две таблицы по номеру телефона, причем в одной из таблиц - номер может быть в разных форматах вида : +79261234567, +7 (926)1234568, 8-926-123-45-67 / 
Во второй таблице номер фиксирован вида : 9261234567

Чтобы показать что наше решение работает создадим 2 таблицы

In [82]:
# таблица выданных кредитов 
credit_list = pd.DataFrame(np.random.randint(0,20,size=(10, 3)), columns=('number','A','B'))
for i in range(len(credit_list)):
  credit_list.loc[i,'number'] = 9261234560 + i
credit_list.head()

,number,A,B
0,9261234560,18,2
1,9261234561,5,9
2,9261234562,3,14
3,9261234563,10,18
4,9261234564,17,17


In [83]:
#таблица заявок
bad_phone = pd.DataFrame(['+79261234567', '+7 (926)1234568', '8-926-123-45-61', '9261234565'], columns=['bad_number'])
bad_phone['C'] = np.random.randint(100,200)
bad_phone

,bad_number,C
0,+79261234567,190
1,+7 (926)1234568,190
2,8-926-123-45-61,190
3,9261234565,190


Для того чтобы убрать лишние символы воспользуемся регулярными выражениями

In [84]:
def good_number(ee):
  result  = re.sub(r'\D*','', ee)
  return result[len(result)-10:11] # оставлем последние 10 цифр из номера
bad_phone['number'] = bad_phone['bad_number'].apply(good_number).astype(int)
bad_phone

,bad_number,C,number
0,+79261234567,190,9261234567
1,+7 (926)1234568,190,9261234568
2,8-926-123-45-61,190,9261234561
3,9261234565,190,9261234565


Склеим таблицы по номеру телефона

In [85]:
last_table = bad_phone[['number','C']].merge(credit_list, how = 'left', on = 'number')
last_table

,number,C,A,B
0,9261234567,190,15,19
1,9261234568,190,9,14
2,9261234561,190,5,9
3,9261234565,190,11,7


##Для автоматизации можно использовать недельный скрипт
который будет по входным параметрам дат( начало и конец недели), брать данные из двух таблиц, приводить номера телефонов к правильному виду и мержить их

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import sys
import getopt
import re
import pandas as pd
import numpy as np
from datetime import datetime
from sqlalchemy import create_engine

if __name__ == "__main__":

    #Задаем входные параметры
    unixOptions = "sdt:edt"
    gnuOptions = ["start_dt=", "end_dt="]
    
    full = sys.argv
    argumentList = full[1:]  # список аргументов, в нашем случае даты
    
    try: # Проверяем входные параметры на соответствие формату
        arguments, values = getopt.getopt(argumentList, unixOptions, gnuOptions)
    except # Прерываем выполнение, если входные параметры некорректны
        getopt.error as err:
        print (str(err))
        sys.exit(2)
    
    start_dt = ''
    end_dt = ''
    for currentArgument, currentValue in arguments:
        if currentArgument in ("-sdt", "--start_dt"):
            start_dt = currentValue
        elif currentArgument in ("-edt", "--end_dt"):
            end_dt = currentValue
    
    # выполняем подключение к базе данных банка
    db_config = {'user': 'my_user',         
                 'pwd': 'my_user_password', 
                 'host': 'localhost',       
                 'port': 1234,              
                 'db': 'bank_rosselhoz'}             
    
    connect = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                     db_config['pwd'],
                                 db_config['host'],
                                 db_config['port'],
                                 db_config['db'])
    
    engine = create_engine(connect)
    
    #Теперь выберем из таблицы в заявками на кредит (bad_phone) только те строки,
    #которые соответствуют этой неделе в столбце data -  между start_dt и end_dt
    query = ''' SELECT *
            FROM bad_phone
            WHERE data::TIMESTAMP BETWEEN '{}'::TIMESTAMP AND '{}'::TIMESTAMP
        '''.format(start_dt, end_dt)
    
    bad_phone = pd.io.sql.read_sql(query, con = engine, index_col = 'data')
    bad_phone['data'] = pd.to_datetime(data_raw['data']) #переводим в формат даты
    
    #функция удаления лишний информации из номера
    def good_number(ee):
      result  = re.sub(r'\D*','', ee)
      return result[len(result)-10:11] # оставлем последние 10 цифр из номера

    bad_phone['number'] = bad_phone['bad_number'].apply(good_number).astype(int)

    # выбираем таблицу с выдаными кредитами
    query = '''SELECT  *
          FROM credit_list
                  '''

    credit_list = pd.io.sql.read_sql(query, con = engine)
    last_table = bad_phone[['number']].merge(credit_list, how = 'left', on = 'number')
    
    # записываем в таблицу last_table - ее длинна будет нам давать сколько всего поступило 
    # заявок за неделю, не нулевые значения в данных - покажут сколько по этим заявкам
    # было выдано кредитов, на какую сумму и так далее.
    last_table.to_sql(name = 'last_table', con = engine, if_exists = 'append', index = False))